# ワーカ間一致率によるクラウドソーシング結果の評価
<a target="_blank" href="https://colab.research.google.com/github/takumi1001/seccamp2024_D2/blob/main/01_Inter-Rater_Agreement.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## 準備

### Google Colabにインストールされていないライブラリを導入する

In [ ]:
!pip install crowd-kit
!pip install krippendorff

### ライブラリをインポートする

In [1]:
import numpy as np
import pandas as pd
import krippendorff

## クリッペンドルフのαを計算する
講義資料で示した２つの場合について，krippendorffライブラリを用いてαを計算し，手計算の結果と一致しているか見てみましょう

![img1](https://github.com/takumi1001/seccamp2024_D2/blob/main/imgs/responses1.png?raw=true)

In [2]:
# krippendorffライブラリでは，ワーカが回答しない場合はnp.nanとする
worker_A = ("クラゲ","クラゲ","イカ","イカ",np.nan,"タコ",np.nan)
worker_B = ("クラゲ","クラゲ",np.nan,"イカ","クラゲ",np.nan,"クラゲ")
worker_C = ("クラゲ",np.nan,"タコ","クラゲ","タコ","タコ","クラゲ")

# 回答をまとめる
responses = (worker_A, worker_B, worker_C)

# 計算する
krippendorff.alpha(reliability_data=responses, level_of_measurement="nominal")

0.4

![img2](https://github.com/takumi1001/seccamp2024_D2/blob/main/imgs/responses2.png?raw=true)

In [3]:
# krippendorffライブラリでは，ワーカが回答しない場合はnp.nanとする
worker_A = ("イカ","クラゲ","イカ","タコ","イカ","タコ",np.nan)
worker_B = ("クラゲ",np.nan,"タコ","イカ","タコ","イカ","クラゲ")
worker_C = ("クラゲ","イカ",np.nan,"クラゲ",np.nan,np.nan,"クラゲ")

# 回答をまとめる
responses = (worker_A, worker_B, worker_C)

# 計算する
krippendorff.alpha(reliability_data=responses, level_of_measurement="nominal")

-0.1607142857142856

## 公開データセットのクリッペンドルフのαを計算してみる
クラウドソーシング企業Toloka社が公開しているデータセット`relevance-2`を用いて，実際のアノテーション結果を対象に実験してみましょう．

In [4]:
from crowdkit.datasets.load_dataset import load_dataset

In [5]:
df, gt = load_dataset('relevance-2')

# 正解データがついているタスクのみを抽出する
## relevance-2には正解データがあるタスクとないタスクの双方が含まれています
## クリッペンドルフのαは正解データのないタスクでも計算できますが，
## メモリ使用量が多いとcolabがクラッシュしてしまうこともあり，正解データのあるデータのみを用います
## 自分のPCでノートブックを動かせる人は，以下の2行をコメントアウトして，データ全体で計算してみてください
df = df.merge(gt, on="task", how="left", validate="m:1")
df = df[df["true_label"].notnull()]

データセットの中身を見てみましょう．
（`true_label`は正解データで，今回は使いません）

In [6]:
df.head()

,worker,task,label,true_label
0,w851,t30685,1,1.0
8,w5879,t7116,1,1.0
14,w3048,t95532,0,1.0
25,w3716,t12806,1,1.0
45,w5843,t90925,0,1.0


データセットは`(ワーカID, タスクID, 回答)`のタプルのリストになっています．

与えられた二つのテキストが，関連している内容`1`かそうでない`0`かを問うタスクです．

In [7]:
df.count()

worker        48268
task          48268
label         48268
true_label    48268
dtype: int64

4万件以上もの回答が含まれている巨大なデータセットです（`gt`がないタスクを含めると，40万件以上あります）．

このデータをkrippendorffライブラリで扱えるように変形します．

In [9]:
responses =  df.pivot_table(index='worker', columns='task', values='label', fill_value=np.nan)

## `df.pivot_table`による変換にはいくつかの制約がある
## 1. 同じワーカが同じタスクに回答していないこと
### `df[df[["worker","task"]].duplicated()].count()` で確認できる
### 同じワーカが同じタスクに複数回回答する場合はクリッペンドルフのαがそもそも計算できない
## 2. ラベルが数値で表現されていること
### `aggfunc`引数を指定し，自分で`df.pivot_table`をカスタムすることで数値以外でも使用できる
## また，`df.pivot_table`はメモリの使用効率が大変に悪い機能なので注意

In [10]:
responses.shape

(7054, 10079)

(ワーカ数,タスク数）の行列に変換されています

計算してみましょう

In [11]:
krippendorff.alpha(reliability_data=responses, level_of_measurement="nominal")

0.2931638815975395

意外に低い値になっています

## データセットにスパムワーカを追加する
このデータセットに悪いふるまいをするワーカを追加してみましょう．

悪いふるまいをするワーカはすべてのタスクにランダムに回答します

このスパムワーカを20人追加します

In [12]:
num_tasks = responses.shape[1] # 総タスク数
num_spam_workers = 20 # スパムワーカ数

In [13]:
spam_workers = np.array([np.random.randint(low=0,high=1+1,size=num_tasks) for _ in range(num_spam_workers)])

In [14]:
spam_workers, spam_workers.shape

(array([[0, 0, 0, ..., 1, 1, 0],
        [1, 0, 0, ..., 1, 1, 1],
        [0, 1, 0, ..., 0, 0, 1],
        ...,
        [0, 1, 1, ..., 0, 1, 0],
        [1, 0, 1, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 1]]),
 (20, 10079))

In [15]:
responses_with_spam = np.concatenate([responses, spam_workers]) # データセットとスパムワーカ結果を結合

In [16]:
krippendorff.alpha(reliability_data=responses_with_spam, level_of_measurement="nominal")

0.007930105458503034

ランダムなふるまいをするスパムワーカの影響で，クリッペンドルフのαが0に近づいたことがわかります．

## 余裕があればやってみよう
 - スパムワーカの数を増減して，クリッペンドルフのαがどう変化するか観察してみましょう
 - `relevance-5`は同じくテキストの関連度を答えるタスクですが，５段階の順序付き回帰問題になっています．`krippendorff.alpha`を`level_of_measurement="ordinal"`として実行し，クリッペンドルフのαを計算してみましょう（`load_dataset('relevance-5')`でダウンロードできます）
 - 自分でクラス分類に対するクリッペンドルフのαを計算する関数を書いてみましょう
 - クラス分類ではない場合のクリッペンドルフのαがどう計算されるのか，調べてみましょう